In [1]:
import torch
import pandas as pd
import sklearn

from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import TrainingArguments, Trainer
from transformers import pipeline

import sys
sys.path.append('../sentence-sentimental')

from dataset.datasets import SentimentalDataset
from metrics.metrics import compute_metrics

from sklearn.datasets import load_iris # 샘플 데이터 로딩
from sklearn.model_selection import train_test_split

/opt/conda/envs/mrc/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
MODEL_NAME = 'klue/roberta-base'
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3).to(device)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'clas

In [3]:
data = pd.read_csv('/opt/ml/finance_sentiment_corpus/finance_data.csv')
data['labels'] = data['labels'].map({'negative':0, 'neutral':1, 'positive':2})

train_encoding = tokenizer(
    data['kor_sentence'].tolist(),
    return_tensors='pt',
    padding=True,
    truncation=True
    )

train_set = SentimentalDataset(train_encoding, data['labels'])

In [4]:
type(data)

pandas.core.frame.DataFrame

In [5]:
import pandas as pd
from sklearn.datasets import load_wine

wine = load_wine()

In [6]:
X = wine['data']
Y = wine['target']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, stratify=Y, random_state=777)

In [8]:
type(X_train)

numpy.ndarray

In [9]:
type(X)

numpy.ndarray

In [10]:
print(len(data['kor_sentence']), len(data['labels']))

4846 4846


In [11]:
type(data['kor_sentence'])

pandas.core.series.Series

In [12]:
data['kor_sentence']

0       Gran에 따르면, 그 회사는 회사가 성장하고 있는 곳이지만, 모든 생산을 러시아로...
1       테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기 위해 10만 평...
2       국제 전자산업 회사인 엘코텍은 탈린 공장에서 수십 명의 직원을 해고했으며, 이전의 ...
3       새로운 생산공장으로 인해 회사는 예상되는 수요 증가를 충족시킬 수 있는 능력을 증가...
4       2009-2012년 회사의 업데이트된 전략에 따르면, Basware는 20% - 4...
                              ...                        
4841    런던 마켓워치 -- 은행주의 반등이 FTSE 100지수의 약세를 상쇄하지 못하면서 ...
4842    린쿠스키아의 맥주 판매량은 416만 리터로 6.5% 감소했으며 카우노 알루스의 맥주...
4843    영업이익은 2007년 68.8 mn에서 35.4 mn으로 떨어졌으며, 선박 판매 이...
4844    페이퍼 부문 순매출은 2008년 2분기 241.1 mn에서 2009년 2분기 221...
4845     핀란드에서의 판매는 1월에 10.5% 감소한 반면, 국외에서의 판매는 17% 감소했다.
Name: kor_sentence, Length: 4846, dtype: object

In [14]:
sentence_train, sentence_test, label_train, label_test = train_test_split(data['kor_sentence'], data['labels'],
                                                                            test_size=0.2, 
                                                                            # shuffle=True, 
                                                                            stratify=data['labels'], # label에 비율을 맞춰서 분리
                                                                            random_state=34)

In [41]:
type(data['kor_sentence'])

pandas.core.series.Series

In [39]:
type(label_train)

pandas.core.series.Series

In [26]:
sentence_train

1828                핀란드 식품업계 기업들은 우크라이나가 제공하는 기회에 관심이 많다.
4809    펜틱+아이넨은 미디어하우스가 제공하는 인터넷 콘텐츠 대부분이 영원히 무료일 수는 없...
1166                    또한 이러한 시장에 대한 6년간의 역사적 분석이 제공됩니다.
277     루키 총리는 핀란드 사본리나에 위치한 키론살미 해협을 가로지르는 철제 구조물의 인도...
4670    2009년 8월 3일 핀란드 미디어 그룹 Ilkka-Yhtyma Oyj(헬: ILK...
                              ...                        
3515    알스트롬 주식거래소 7.2.2.27 10시 30분에 발표 알스트롬 주식회사의 총 5...
4503           라우트 대변인은 "다른 인력에 대한 적응 조치는 당분간 적절하다"고 말했다.
434     주식수 증가는 2006년 스톡옵션제도에 따라 회사 경영에 부여된 주식선택권에 따른 ...
3212         개발사업단도 지난해 11월 대비 개발사업단가를 3분의 1가량 낮추겠다고 밝혔다.
3773                    이 건물에는 제품 개발 및 테스트 실험실이 들어설 예정이다.
Name: kor_sentence, Length: 3876, dtype: object

In [16]:
train_encoding = tokenizer(
    sentence_train.tolist(),
    return_tensors='pt',
    padding=True,
    truncation=True
    )

test_encoding = tokenizer(
    sentence_test.tolist(),
    return_tensors='pt',
    padding=True,
    truncation=True
    )

In [17]:
train_set = SentimentalDataset(train_encoding, label_train.reset_index(drop=True))
test_set = SentimentalDataset(test_encoding, label_test.reset_index(drop=True))

In [18]:
training_args = TrainingArguments(
        output_dir = './outputs',
        logging_steps = 50,
        num_train_epochs = 1,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        fp16=True
    )

trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_set,
        eval_dataset=test_set,
        compute_metrics=compute_metrics
    )


In [24]:
!wandb offline

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/bin/bash: wandb: command not found


In [25]:

print('---train start---')
trainer.train()

Step,Training Loss


AttributeError: module 'wandb' has no attribute 'log'

In [26]:
sentence_train

1828                핀란드 식품업계 기업들은 우크라이나가 제공하는 기회에 관심이 많다.
4809    펜틱+아이넨은 미디어하우스가 제공하는 인터넷 콘텐츠 대부분이 영원히 무료일 수는 없...
1166                    또한 이러한 시장에 대한 6년간의 역사적 분석이 제공됩니다.
277     루키 총리는 핀란드 사본리나에 위치한 키론살미 해협을 가로지르는 철제 구조물의 인도...
4670    2009년 8월 3일 핀란드 미디어 그룹 Ilkka-Yhtyma Oyj(헬: ILK...
                              ...                        
3515    알스트롬 주식거래소 7.2.2.27 10시 30분에 발표 알스트롬 주식회사의 총 5...
4503           라우트 대변인은 "다른 인력에 대한 적응 조치는 당분간 적절하다"고 말했다.
434     주식수 증가는 2006년 스톡옵션제도에 따라 회사 경영에 부여된 주식선택권에 따른 ...
3212         개발사업단도 지난해 11월 대비 개발사업단가를 3분의 1가량 낮추겠다고 밝혔다.
3773                    이 건물에는 제품 개발 및 테스트 실험실이 들어설 예정이다.
Name: kor_sentence, Length: 3876, dtype: object

In [8]:
type(data)

numpy.ndarray

In [9]:
type(train_set)

dataset.datasets.SentimentalDataset